
# Project Objectives
---

<div style="text-align: justify"> 
Provider Fraud is one of the biggest problems facing Medicare. According to the government, the total Medicare spending increased exponentially due to frauds in Medicare claims. Healthcare fraud is an organized crime which involves peers of providers, physicians, beneficiaries acting together to make fraud claims. Rigorous analysis of Medicare data has yielded many physicians who indulge in fraud. They adopt ways in which an ambiguous diagnosis code is used to adopt costliest procedures and drugs. Insurance companies are the most vulnerable institutions impacted due to these bad practices. Due to this reason, insurance companies increased their insurance premiums and as result healthcare is becoming costly matter day by day.</div>

Healthcare fraud and abuse take many forms. Some of the most common types of frauds by providers are:

- Billing for services that were not provided.

- Duplicate submission of a claim for the same service.

- Misrepresenting the service provided.

- Charging for a more complex or expensive service than was actually provided.

- Billing for a covered service when the service actually provided was not covered.

    


# Problem Statement
---
<div style="text-align: justify">The goal of this project is to " predict the potentially fraudulent providers " based on the claims filed by them. Along with this, we will also discover important variables helpful in detecting the behaviour of potentially fraud providers. Further, we will study fraudulent patterns in the provider's claims to understand the future behaviour of providers.</div>


# Introduction to the Dataset
---
<div style="text-align: justify">For the purpose of this project, we are considering Inpatient claims, Outpatient claims and Beneficiary details of each provider. Lets s see their details :</div>
    
**1. Inpatient Data**: This data provides insights about the claims filed for those patients who are admitted in the hospitals. It also provides additional details like their admission and discharge dates and admit d diagnosis code.

**2. Outpatient Data**: This data provides details about the claims filed for those patients who visit hospitals and not admitted in it.
    
**3. Beneficiary Details Data**: This data contains beneficiary KYC details like health conditions,region they belong to etc.

In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile

# 1. Importing Data

In [2]:
provider_data=pd.read_csv(r"Data\provider_fraud_detection_data.csv")
provider_data

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No
...,...,...
5405,PRV57759,No
5406,PRV57760,No
5407,PRV57761,No
5408,PRV57762,No


In [3]:
data_list=['outpatientdata','inpatientdata','beneficiarydata']
dataframes=[]
for i in data_list:
    zipfile_path=f'Data/{i}.zip'
    zf=zipfile.ZipFile(zipfile_path)
    for file in zf.namelist():
        print (f"files in {zipfile_path} : {file}")
        df=pd.read_csv(zf.open(file))
        dataframes.append(df)

files in Data/outpatientdata.zip : Train_Outpatientdata-1542865627584.csv
files in Data/inpatientdata.zip : Train_Inpatientdata-1542865627584.csv
files in Data/beneficiarydata.zip : Train_Beneficiarydata-1542865627584.csv


In [4]:
outpatient_data=dataframes[0]
inpatient_data=dataframes[1]
beneficiary_data=dataframes[2]

# 2. Understand the Data

### a. Provider Data

In [32]:
provider_data.head()

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No


In [33]:
provider_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Provider        5410 non-null   object
 1   PotentialFraud  5410 non-null   object
dtypes: object(2)
memory usage: 84.7+ KB


In [34]:
provider_data.nunique()

Provider          5410
PotentialFraud       2
dtype: int64

In [35]:
x=provider_data.groupby(['PotentialFraud']).count()
x

,Provider
PotentialFraud,
No,4904
Yes,506


#### There is 506 potential fraud providers. There is a possibility that they are either filing correct or incorrect bills.

### b. Outpatient Data

In [36]:
outpatient_data.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,NaN,NaN,6115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,NaN,NaN,2723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947


In [51]:
print(f"Columns: {len(outpatient_data.columns)} \n   Rows: {len(outpatient_data)}")

Columns: 27 
   Rows: 517737


In [52]:
outpatient_data_info=pd.DataFrame(data=outpatient_data.columns, 
                                 columns=['Column Name'])

In [53]:
outpatient_data_info['Data Type']=outpatient_data.dtypes.values
outpatient_data_info['No.of Unique Values']=outpatient_data.nunique().values
outpatient_data_info['Count of Missing Nos']=outpatient_data.isna().sum().values
outpatient_data_info['Percent of Missing Nos']=(round(outpatient_data.isna().sum()/len(outpatient_data)*100)).values

In [54]:
outpatient_data_info

,Column Name,Data Type,No.of Unique Values,Count of Missing Nos,Percent of Missing Nos
0,BeneID,object,133980,0,0.0
1,ClaimID,object,517737,0,0.0
2,ClaimStartDt,object,385,0,0.0
3,ClaimEndDt,object,366,0,0.0
4,Provider,object,5012,0,0.0
5,InscClaimAmtReimbursed,int64,342,0,0.0
6,AttendingPhysician,object,74109,1396,0.0
7,OperatingPhysician,object,28532,427120,82.0
8,OtherPhysician,object,44388,322691,62.0
9,ClmDiagnosisCode_1,object,10354,10453,2.0


### c. Inpatient Data

In [55]:
inpatient_data.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN


In [57]:
print(f"Columns: {len(inpatient_data.columns)} \n   Rows: {len(inpatient_data)}")

Columns: 30 
   Rows: 40474


In [58]:
inpatient_data_info=pd.DataFrame(data=inpatient_data.columns, 
                                 columns=['Column Name'])

In [59]:
inpatient_data_info['Data Type']= inpatient_data.dtypes.values
inpatient_data_info['No.of Unique Values']= inpatient_data.nunique().values
inpatient_data_info['Count of Missing Nos']= inpatient_data.isna().sum().values
inpatient_data_info['Percent of Missing Nos']=(round(inpatient_data.isna().sum()/len(inpatient_data)*100)).values

In [60]:
inpatient_data_info

,Column Name,Data Type,No.of Unique Values,Count of Missing Nos,Percent of Missing Nos
0,BeneID,object,31289,0,0.0
1,ClaimID,object,40474,0,0.0
2,ClaimStartDt,object,398,0,0.0
3,ClaimEndDt,object,365,0,0.0
4,Provider,object,2092,0,0.0
5,InscClaimAmtReimbursed,int64,146,0,0.0
6,AttendingPhysician,object,11604,112,0.0
7,OperatingPhysician,object,8287,16644,41.0
8,OtherPhysician,object,2877,35784,88.0
9,AdmissionDt,object,398,0,0.0


### d. Finding columns shared by inpatient and outpatient data 

In [67]:
# Finding Common columns
a = inpatient_data.columns.intersection(outpatient_data.columns)
print(len(a))


27


In [69]:
inpatient_data.shape

(40474, 30)

In [79]:
print("The columns which are only in the inpatient data:")
for i in inpatient_data.columns:
    #print (i)
    if i not in a:
        print(f"  {i}")

The columns which are only in the inpatient data:
  AdmissionDt
  DischargeDt
  DiagnosisGroupCode


####  There is claim start date and claim end date in the inpatient data. We need to check whether the 
    1. "ClaimStartDt=AdmissionDt" and 
    2. "ClaimEndDt=DischargeDt"

In [91]:
(inpatient_data['ClaimStartDt']==inpatient_data['AdmissionDt'])

0        True
1        True
2        True
3        True
4        True
         ... 
40469    True
40470    True
40471    True
40472    True
40473    True
Length: 40474, dtype: bool

In [92]:
(inpatient_data['ClaimStartDt']==inpatient_data['AdmissionDt']).sum()

40442

In [102]:
x=pd.DataFrame(inpatient_data.loc[inpatient_data['ClaimStartDt']!=inpatient_data['AdmissionDt']]['ClaimStartDt'])

In [105]:
x['AdmissionDt']=inpatient_data.loc[inpatient_data['ClaimStartDt']!=inpatient_data['AdmissionDt']]['AdmissionDt']

In [106]:
x

,ClaimStartDt,AdmissionDt
2163,2009-02-10,2009-02-08
4422,2009-10-02,2009-09-29
5400,2009-09-12,2009-09-09
5647,2009-03-07,2009-03-02
5869,2009-07-10,2009-07-06
7828,2009-04-20,2009-04-16
7869,2009-11-01,2009-10-24
9051,2009-11-26,2009-11-18
9215,2009-01-11,2009-01-06
11103,2009-03-28,2009-03-26


### e. Beneficiary Details Data

In [19]:
beneficiary_data.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,...,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,...,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,...,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,...,2,1,2,2,2,2,0,0,1790,1200


In [20]:
f"The inpatient data has {len(beneficiary_data.columns)} columns and {len(beneficiary_data)} rows"

'The inpatient data has 25 columns and 138556 rows'

In [21]:
beneficiary_data_info=pd.DataFrame(data=beneficiary_data.columns, 
                                 columns=['Column Name'])

In [22]:
beneficiary_data_info['Data Type']= beneficiary_data.dtypes.values
beneficiary_data_info['No.of Unique Values']= beneficiary_data.nunique().values
beneficiary_data_info['Count of Missing Nos']= beneficiary_data.isna().sum().values
beneficiary_data_info['Percent of Missing Nos']=(round(beneficiary_data.isna().sum()/len(beneficiary_data)*100)).values

In [23]:
beneficiary_data_info

,Column Name,Data Type,No.of Unique Values,Count of Missing Nos,Percent of Missing Nos
0,BeneID,object,138556,0,0.0
1,DOB,object,900,0,0.0
2,DOD,object,11,137135,99.0
3,Gender,int64,2,0,0.0
4,Race,int64,4,0,0.0
5,RenalDiseaseIndicator,object,2,0,0.0
6,State,int64,52,0,0.0
7,County,int64,314,0,0.0
8,NoOfMonths_PartACov,int64,13,0,0.0
9,NoOfMonths_PartBCov,int64,13,0,0.0


# 3. Data Preprocessing

### Merging Inpatient and Outpatient Data Columns

In [24]:
len(set(outpatient_data.columns).intersection(set(inpatient_data.columns)))

27

In [25]:
len(inpatient_data.columns)

30

In [26]:
in_out_patient_data=outpatient_data.append(inpatient_data, ignore_index = True)

In [27]:
in_out_patient_data.shape

(558211, 30)

In [30]:
outpatient_data.shape[0]+inpatient_data.shape[0]

558211

### Joining provider details (Fraudulent or not)

In [31]:
in_out_patient_data=pd.merge(in_out_patient_data,provider_data,on='Provider')
in_out_patient_data

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode,AdmissionDt,DischargeDt,DiagnosisGroupCode,PotentialFraud
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,NaN,NaN,NaN,NaN,0.0,56409,NaN,NaN,NaN,Yes
1,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Yes
2,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,NaN,NaN,NaN,NaN,0.0,71947,NaN,NaN,NaN,Yes
3,BENE11004,CLM173224,2009-02-03,2009-02-03,PRV56011,20,PHY339887,NaN,NaN,20381,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Yes
4,BENE11004,CLM224741,2009-03-03,2009-03-03,PRV56011,40,PHY345721,NaN,NaN,V6546,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,BENE149280,CLM68702,2009-09-19,2009-09-24,PRV54054,5000,PHY405930,NaN,NaN,486,...,NaN,NaN,NaN,NaN,1068.0,486,2009-09-19,2009-09-24,183,No
558207,BENE151892,CLM31767,2008-12-31,2009-01-02,PRV51136,20000,PHY361876,PHY361876,NaN,44481,...,NaN,NaN,NaN,NaN,1068.0,7295,2008-12-31,2009-01-02,312,No
558208,BENE154375,CLM39644,2009-02-22,2009-02-25,PRV51977,8000,PHY323189,PHY349527,NaN,44024,...,NaN,NaN,NaN,NaN,1068.0,44023,2009-02-22,2009-02-25,314,No
558209,BENE156181,CLM57017,2009-06-24,2009-07-01,PRV55706,12000,PHY319644,PHY319644,PHY403799,78079,...,NaN,NaN,NaN,NaN,1068.0,78097,2009-06-24,2009-07-01,940,No


### Cleaning Beneficiery Details

In [59]:
beneficiary_data

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,...,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,...,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,...,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,...,2,1,2,2,2,2,0,0,1790,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138551,BENE159194,1939-07-01,NaN,1,1,0,39,140,12,12,...,2,2,2,2,2,2,0,0,430,460
138552,BENE159195,1938-12-01,NaN,2,1,0,49,530,12,12,...,2,1,2,2,2,2,0,0,880,100
138553,BENE159196,1916-06-01,NaN,2,1,0,6,150,12,12,...,1,1,1,2,2,2,2000,1068,3240,1390
138554,BENE159197,1930-01-01,NaN,1,1,0,16,560,12,12,...,2,2,1,2,2,2,0,0,2650,10


In [65]:
beneficiary_data['DOB'] =  pd.to_datetime(beneficiary_data['DOB'], format='%Y-%m-%d')
beneficiary_data['DOD'] =  pd.to_datetime(beneficiary_data['DOD'], format='%Y-%m-%d')
beneficiary_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138556 entries, 0 to 138555
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   BeneID                           138556 non-null  object        
 1   DOB                              138556 non-null  datetime64[ns]
 2   DOD                              1421 non-null    datetime64[ns]
 3   Gender                           138556 non-null  int64         
 4   Race                             138556 non-null  int64         
 5   RenalDiseaseIndicator            138556 non-null  object        
 6   State                            138556 non-null  int64         
 7   County                           138556 non-null  int64         
 8   NoOfMonths_PartACov              138556 non-null  int64         
 9   NoOfMonths_PartBCov              138556 non-null  int64         
 10  ChronicCond_Alzheimer            138556 non-